* Lista das Debêntures: https://data.anbima.com.br/debentures?view=precos

---
Observações da tabela:
* (*) O papel tem cláusula de Resgate ou Amortização Antecipado.
* (**) A cláusula já está no período de exercício.
* (#) Condição atual de negociação em “combo” dessas emissões.
* Algumas séries de debêntures não têm seus preços divulgados, em função de características não padronizadas ou incertezas sobre os eventos financeiros.  
* *** O PUPAR adotado para os ativos LIGHA5 LIGHB4 LIGHB6 LIGHC3 LIGHC6 LIGHD2 LIGHD3 considera os juros acumulados devidos e não pagos em 17/04, até que novas condições sejam divulgadas pelo emissor.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
def debentures(path: str, sheet_name: str):
    """
    Formata o arquivo da Anbima das debêntures listadas no mercado secundário.

    Parameters
    ----------
    path: str 
        Caminho do arquivo excel.
    sheet_name: str 
        Nome da aba do arquivo excel.

    Returns
    -------
    df: pd.DataFrame
        Df das debêntures.
    
    """
    # Debêntures indexados ao IPCA spread
    df = pd.read_excel(io=path, sheet_name=sheet_name, header=7, skipfooter=6)

    # Renomeando as colunas
    df = df.rename(columns={
        'Código': 'Codigo',
        'Repac./  Venc.': 'Vencimento',
        'Intervalo Indicativo': 'Intervalo Indicativo Min',
        'Unnamed: 9': 'Intervalo Indicativo Max'
    })

    # Retirando a 1º linha que só tem o 'Min' e 'Max' da coluna 'Intervalo Indicativo'
    df = df[1:]

    # Condição para separar a coluna 'Índice/ Correção'
    if (sheet_name == 'IPCA_SPREAD') or (sheet_name == 'DI_SPREAD'):
        # Separando a coluna 'Índice/ Correção' em duas colunas 'Indexador' e 'Taxa' -> de 'IPCA + 7,0718%' para 'IPCA' e '7,0718' / de 'DI + 4,35%' para 'DI' e '4,35'
        df[['Indexador', 'Taxa']] = df['Índice/ Correção'].str.extract(r'(.+) \+ (.+)%')
    
    else:
        # Separando a coluna 'Índice/ Correção' em duas colunas 'Taxa' e 'Indexador' -> de '108% do DI' para '108%' e 'DI'
        df[['Taxa', 'Indexador']] = df['Índice/ Correção'].str.split(' do', n=1, expand=True)

        # Removendo o símbolo '%' da coluna 'Taxa'
        df['Taxa'] = df['Taxa'].str.replace('%', '')


    # Substituindo a ',' por '.' na coluna 'Taxa'
    df['Taxa'] = df['Taxa'].str.replace(',', '.')

    # Substituindo a string 'N/D' por NaN
    df['Duration'] = df['Duration'].replace('N/D', np.nan)

    # Transformando os dtypes das colunas
    df['Nome'] = df['Nome'].astype(str)
    df['Codigo'] = df['Codigo'].astype(str)
    df['Taxa'] = df['Taxa'].astype(float)
    df['Duration'] = df['Duration'].astype(float)
    df['Vencimento'] = pd.to_datetime(df['Vencimento'], dayfirst=True)

    # Selecionando apenas as principais colunas
    df = df[['Codigo', 'Nome', 'Vencimento', 'Duration', 'Indexador', 'Taxa']]

    return df

In [ ]:
nome_arquivo = 'd26fev25'

debentures_ipca = debentures(
    path=f'C://B3//analise-fundamentalista//debentures//{nome_arquivo}.xls', 
    sheet_name='IPCA_SPREAD'
)

debentures_di = debentures(
    path=f'C://B3//analise-fundamentalista//debentures//{nome_arquivo}.xls', 
    sheet_name='DI_SPREAD'
)

debentures_di_perc = debentures(
    path=f'C://B3//analise-fundamentalista//debentures//{nome_arquivo}.xls', 
    sheet_name='DI_PERCENTUAL'
)

# Removendo o espaço na frente da string " DI" -> "DI"
debentures_di_perc['Indexador'] = debentures_di_perc['Indexador'].str.strip()

In [ ]:
# Ordenando o df das debêntures com a maior taxa 
debentures_ipca.sort_values(by='Taxa', ascending=False).head(30)

In [ ]:
# Ordenando o df das debêntures com a maior taxa 
debentures_di.sort_values(by='Taxa', ascending=False).head(20)

In [ ]:
# Ordenando o df das debêntures com a maior taxa 
debentures_di_perc.sort_values(by='Taxa', ascending=False)

In [ ]:
# Concatenando os três dfs em apenas um df
df_debentures = pd.concat([debentures_ipca, debentures_di, debentures_di_perc])
df_debentures 

## Debêntures dos principais setores da Bolsa

### Energia Elétrica

In [ ]:
# Geração
nomes_empresas = ['ENGIE', 'ELETROBRAS', 'AES TIETE', 'AES BRASIL', 'OMEGA', 'ENEVA']
filt = df_debentures['Nome'].str.contains('|'.join(nomes_empresas))
df_debentures.loc[filt].sort_values(by='Nome')

In [ ]:
# Distribuição
nomes_empresas = ['CEMIG', 'COPEL', 'EQUATORIAL', 'NEO', 'LIGHT']
filt = df_debentures['Nome'].str.contains('|'.join(nomes_empresas))
df_debentures.loc[filt].sort_values(by='Nome')

In [ ]:
# Transmissão
nomes_empresas = ['ALIANÇA', 'ALUPAR', 'ELÉTRICA PAULISTA']
filt = df_debentures['Nome'].str.contains('|'.join(nomes_empresas))
df_debentures.loc[filt].sort_values(by='Nome')

### Bancos

In [ ]:
nome_empresa = 'ITAUSA'
filt = df_debentures['Nome'].str.contains(nome_empresa)
df_debentures.loc[filt].sort_values(by='Nome')

### Varejo

In [ ]:
# Eletrodomésticos
nomes_empresas = ['MAGAZINE', 'VAREJO']
filt = df_debentures['Nome'].str.contains('|'.join(nomes_empresas))
df_debentures.loc[filt].sort_values(by='Nome')

In [ ]:
# Roupa
nomes_empresas = ['RENNER', 'GUARARAPES', 'C E A MODAS', 'SBF']
filt = df_debentures['Nome'].str.contains('|'.join(nomes_empresas))
df_debentures.loc[filt].sort_values(by='Nome')

### Papel e Celulose

In [ ]:
nomes_empresas = ['KLABIN', 'SUZANO', 'IRANI']
filt = df_debentures['Nome'].str.contains('|'.join(nomes_empresas))
df_debentures.loc[filt].sort_values(by='Nome')

### Aluguél de Carros

In [ ]:
nomes_empresas = ['LOCALIZA', 'MOVIDA', 'VAMOS LOCACAO']
filt = df_debentures['Nome'].str.contains('|'.join(nomes_empresas))
df_debentures.loc[filt].sort_values(by='Nome')

### Petróleo 

In [ ]:
nomes_empresas = ['PETROBRAS', 'PETRO RIO']
filt = df_debentures['Nome'].str.contains('|'.join(nomes_empresas))
df_debentures.loc[filt].sort_values(by='Nome')

### Mineração

In [ ]:
nomes_empresas = ['VALE S/A', 'CSN MINERACAO S.A', 'AURA ALMAS']
filt = df_debentures['Nome'].str.contains('|'.join(nomes_empresas))
df_debentures.loc[filt].sort_values(by='Nome')

### Água e Saneamento Básico

In [ ]:
nomes_empresas = ['SANEPAR', 'SABESP', 'COPASA', 'ORIZON', 'AMBIPAR']
filt = df_debentures['Nome'].str.contains('|'.join(nomes_empresas))
df_debentures.loc[filt].sort_values(by='Nome')

### Shopping

In [ ]:
nomes_empresas = ['IGUATEMI', 'MULTIPLAN', 'ALIANSCE SONAE', 'BR MALLS PARTICIPAÇÕES S/A']
filt = df_debentures['Nome'].str.contains('|'.join(nomes_empresas))
df_debentures.loc[filt].sort_values(by='Nome')

### Saúde

In [ ]:
nomes_empresas = ['HOSPITAL MATER DEI S.A']
filt = df_debentures['Nome'].str.contains('|'.join(nomes_empresas))
df_debentures.loc[filt].sort_values(by='Nome')


In [ ]:
# Hospitais
nomes_empresas = ['HOSPITAL MATER DEI S.A', 'ONCOCLINICAS']
filt = df_debentures['Nome'].str.contains('|'.join(nomes_empresas))
df_debentures.loc[filt].sort_values(by='Nome')

In [ ]:
# Diagonóstico por Imagem
nomes_empresas = ['FLEURY', 'DIAGNÓSTICOS DA AMÉRICA']
filt = df_debentures['Nome'].str.contains('|'.join(nomes_empresas))
df_debentures.loc[filt].sort_values(by='Nome')

In [ ]:
# Plano de Saúde
nomes_empresas = ['HAPVIDA', 'QUALICORP', 'ODONTOPREV']
filt = df_debentures['Nome'].str.contains('|'.join(nomes_empresas))
df_debentures.loc[filt].sort_values(by='Nome')

In [ ]:
# Medicamentos e Outros Produtos
nomes_empresas = ['RAIA DROGASIL', 'DIMED', 'HYPERA']
filt = df_debentures['Nome'].str.contains('|'.join(nomes_empresas))
df_debentures.loc[filt].sort_values(by='Nome')

## Debêntures dos Fundos de Previdência

### Itau Prev High Yield

In [ ]:
# Ano e mês do arquivo aberto
ano = '2025'
mes = '06'

# Lendo o arquivo excel que contém o portfólio de debêntures
df_itau_hy_prev = pd.read_excel(
    io=f'C://B3//composicao-fundos-de-investimentos//fundos_previdencia//itau_hy//fundo_itau_prev_hy_{ano}{mes}.xlsx',
    skipfooter=1
)

# Quais debêntures que estão no portfólio do Itau Prev HY estão listados na ANBIMA
df_itau_hy_prev_anbima = df_debentures[df_debentures['Codigo'].isin(df_itau_hy_prev['CD_ATIVO'])]

# Debêntures listadas na ANBIMA do portfólio do Itau Prev HY com indexador DI
filt_di = (df_itau_hy_prev_anbima['Indexador'] == 'DI')
df_itau_hy_prev_anbima_di = df_itau_hy_prev_anbima.loc[filt_di]

# Debêntures listadas na ANBIMA do portfólio do Itau Prev HY com indexador IPCA
filt_ipca = (df_itau_hy_prev_anbima['Indexador'] == 'IPCA')
df_itau_hy_prev_anbima_ipca = df_itau_hy_prev_anbima.loc[filt_ipca]

# Debêntures que não estão listadas na ANBIMA
df_itau_hy_prev_missing = df_itau_hy_prev[~df_itau_hy_prev['CD_ATIVO'].isin(df_debentures['Codigo'])]
# Ordenando em ordem alfabética pelo ticker da debênture
df_itau_hy_prev_missing = df_itau_hy_prev_missing.sort_values('CD_ATIVO')

# Número de debêntures no portfólio do Itau Prev HY
print('-'*100)
print(f'O número total de debêntures no portfólio do Itau Prev HY é de {len(df_itau_hy_prev)}.')
print(f'Dessas {len(df_itau_hy_prev)} debêntures, apenas {len(df_itau_hy_prev_anbima)} estão listadas no arquivo da ANBIMA.')
print(f'Das {len(df_itau_hy_prev_anbima)} debêntures listadas:')
print(f' - {len(df_itau_hy_prev_anbima_di)} debêntures estão indexadas ao DI;')
print(f' - {len(df_itau_hy_prev_anbima_ipca)} debêntures estão indexadas ao IPCA.')
print(f'O número total de debêntures no portfólio do Itau Prev HY que não estão listadas na ANBIMA são de {len(df_itau_hy_prev_missing)}.')
print('-'*100)

In [ ]:
# Transformando o df das debêntures que não estão listadas em um arquivo excel
with pd.ExcelWriter(f'C://B3//analise-fundamentalista//debentures//fundos_prev//itau_hy//itau_hy_prev_debentures_{ano}{mes}.xlsx') as writer:
    df_itau_hy_prev_anbima.to_excel(writer, sheet_name='deb_anbima')
    df_itau_hy_prev_anbima_di.to_excel(writer, sheet_name='deb_di_anbima')
    df_itau_hy_prev_anbima_ipca.to_excel(writer, sheet_name='deb_ipca_anbima')
    df_itau_hy_prev_missing.to_excel(writer, sheet_name='deb_nao_anbima')

### Itau Person Kinea CP

In [ ]:
# Ano e mês do arquivo aberto
ano = '2025'
mes = '06'

# Lendo o arquivo excel que contém o portfólio de debêntures
df_itau_kinea_prev = pd.read_excel(
    io=f'C://B3//composicao-fundos-de-investimentos//fundos_previdencia//itau_person_kinea//fundo_itau_person_kinea_{ano}{mes}.xlsx',
    skipfooter=1
)

# Quais debêntures que estão no portfólio do Itau Kinea Prev estão listados na ANBIMA
df_itau_kinea_prev_anbima = df_debentures[df_debentures['Codigo'].isin(df_itau_kinea_prev['CD_ATIVO'])]

# Debêntures listadas na ANBIMA do portfólio do Itau Kinea Prev com indexador DI
filt_di = (df_itau_kinea_prev_anbima['Indexador'] == 'DI')
df_itau_kinea_prev_anbima_di = df_itau_kinea_prev_anbima.loc[filt_di]

# Debêntures listadas na ANBIMA do portfólio do Itau Kinea Prev com indexador IPCA
filt_ipca = (df_itau_kinea_prev_anbima['Indexador'] == 'IPCA')
df_itau_kinea_prev_anbima_ipca = df_itau_kinea_prev_anbima.loc[filt_ipca]

# Debêntures que não estão listadas na ANBIMA
df_itau_kinea_prev_missing = df_itau_kinea_prev[~df_itau_kinea_prev['CD_ATIVO'].isin(df_debentures['Codigo'])]
# Ordenando em ordem alfabética pelo ticker da debênture
df_itau_kinea_prev_missing = df_itau_kinea_prev_missing.sort_values('CD_ATIVO')

# Número de debêntures no portfólio do Itau Kinea Prev
print('-'*100)
print(f'O número total de debêntures no portfólio do Itau Kinea Prev é de {len(df_itau_kinea_prev)}.')
print(f'Dessas {len(df_itau_kinea_prev)} debêntures, apenas {len(df_itau_kinea_prev_anbima)} estão listadas no arquivo da ANBIMA.')
print(f'Das {len(df_itau_kinea_prev_anbima)} debêntures listadas:')
print(f' - {len(df_itau_kinea_prev_anbima_di)} debêntures estão indexadas ao DI;')
print(f' - {len(df_itau_kinea_prev_anbima_ipca)} debêntures estão indexadas ao IPCA.')
print(f'O número total de debêntures no portfólio do Itau Kinea Prev que não estão listadas na ANBIMA são de {len(df_itau_kinea_prev_missing)}.')
print('-'*100)

In [ ]:
# Transformando o df das debêntures que não estão listadas em um arquivo excel
with pd.ExcelWriter(f'C://B3//analise-fundamentalista//debentures//fundos_prev//itau_person_kinea//itau_kinea_prev_debentures_{ano}{mes}.xlsx') as writer:
    df_itau_kinea_prev_anbima.to_excel(writer, sheet_name='deb_anbima')
    df_itau_kinea_prev_anbima_di.to_excel(writer, sheet_name='deb_di_anbima')
    df_itau_kinea_prev_anbima_ipca.to_excel(writer, sheet_name='deb_ipca_anbima')
    df_itau_kinea_prev_missing.to_excel(writer, sheet_name='deb_nao_anbima')

### Comparação de portfólio de debêntures dos fundos de previdência

In [ ]:
# Portfolio do fundo Itau Prev High Yield
df_itau_hy_prev_portfolio_anbima = pd.read_excel(
    io=f'C://B3//analise-fundamentalista//debentures//fundos_prev//itau_hy//itau_hy_prev_debentures_{ano}{mes}.xlsx',
    sheet_name='deb_anbima'
)

df_itau_hy_prev_portfolio_nao_anbima = pd.read_excel(
    io=f'C://B3//analise-fundamentalista//debentures//fundos_prev//itau_hy//itau_hy_prev_debentures_{ano}{mes}.xlsx',
    sheet_name='deb_nao_anbima'
)

# Portfolio do fundo Itau Kinea Prev
df_itau_kinea_prev_portfolio_anbima = pd.read_excel(
    io=f'C://B3//analise-fundamentalista//debentures//fundos_prev//itau_person_kinea//itau_kinea_prev_debentures_{ano}{mes}.xlsx',
    sheet_name='deb_anbima'
)

df_itau_kinea_prev_portfolio_nao_anbima = pd.read_excel(
    io=f'C://B3//analise-fundamentalista//debentures//fundos_prev//itau_person_kinea//itau_kinea_prev_debentures_{ano}{mes}.xlsx',
    sheet_name='deb_nao_anbima'
)

# Encontrando os tickers comuns
tickers_comuns_anbima = df_itau_hy_prev_portfolio_anbima[df_itau_hy_prev_portfolio_anbima['Codigo'].isin(df_itau_kinea_prev_portfolio_anbima['Codigo'])]
tickers_comuns_nao_anbima = df_itau_hy_prev_portfolio_nao_anbima[df_itau_hy_prev_portfolio_nao_anbima['CD_ATIVO'].isin(df_itau_kinea_prev_portfolio_nao_anbima['CD_ATIVO'])]

# Contando o número de tickers comuns
numero_tickers_comuns_anbima = tickers_comuns_anbima['Codigo'].nunique()
numero_tickers_comuns_nao_anbima = tickers_comuns_nao_anbima['CD_ATIVO'].nunique()

# Lista dos tickers em comuns
tickers_comuns_anbima = list(tickers_comuns_anbima['Codigo'].unique())
tickers_comuns_nao_anbima = list(tickers_comuns_nao_anbima['CD_ATIVO'].unique())

print('='*60)
print(f'Comparação dos portfólio debêntures ANBIMA - {ano}{mes}')
print('='*60)
print(f'Itau Prev High Yield - número de debêntures ANBIMA: {len(df_itau_hy_prev_portfolio_anbima)}.')
print(f'Itau Kinea Prev - número de debêntures ANBIMA: {len(df_itau_kinea_prev_portfolio_anbima)}.')
print(f'Número de debêntures semelhantes ANBIMA: {numero_tickers_comuns_anbima}.')
print('='*60)
print(tickers_comuns_anbima)
print('='*60)
print('='*60)
print(f'Comparação dos portfólio debêntures não ANBIMA - {ano}{mes}')
print('='*60)
print(f'Itau Prev High Yield - número de debêntures não ANBIMA: {len(df_itau_hy_prev_portfolio_nao_anbima)}.')
print(f'Itau Kinea Prev - número de debêntures não ANBIMA: {len(df_itau_kinea_prev_portfolio_nao_anbima)}.')
print(f'Número de debêntures semelhantes não ANBIMA: {numero_tickers_comuns_nao_anbima}.')
print('='*60)
print(tickers_comuns_nao_anbima)
print('='*60)